<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Cubic_Crystal_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs the jupyter-notebook given [here](https://github.com/materialsvirtuallab/m3gnet/blob/main/examples/Cubic%20Crystal%20Test.ipynb) on Colab.

In [1]:
!pip install m3gnet alignn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.8/973.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
import traceback
import warnings

import numpy as np
import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
#from pymatgen.ext.matproj import MPRester
#Run the cell twice to avoid MPRester error
from pymatgen.ext.matproj import _MPResterLegacy as MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")
    warnings.filterwarnings("ignore", category=category, module="pymatgen")

In [4]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]

In [5]:
data

,Material,Lattice constant (Å),Crystal structure,Ref.
0,C (diamond),3.567,Diamond (FCC),[7]
1,C (graphite),a = 2.461 c = 6.708,Hexagonal,NaN
2,Si,5.431020511,Diamond (FCC),[8][9]
3,Ge,5.658,Diamond (FCC),[8]
4,AlAs,5.6605,Zinc blende (FCC),[8]
...,...,...,...,...
86,CaVO3,3.767,Cubic perovskite,[15]
87,BaMnO3,a = 5.673 c = 4.71,Hexagonal,[15]
88,CaMnO3,a = 5.27 b = 5.275 c = 7.464,Orthorhombic perovskite,[15]
89,SrRuO3,a = 5.53 b = 5.57 c = 7.85,Orthorhombic perovskite,[15]


In [6]:

data = data[
    ~data["Crystal structure"].isin(
        ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorhombic", "Tetragonal perovskite", "Orthorhombic perovskite"]
    )
]
data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)

data["a (Å)"] = data["a (Å)"].map(float)


additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""
def add_new(str_, structure_type, df):
    tokens = str_.split()
    new_crystals = []
    for i in range(int(len(tokens) / 3)):
        el = tokens[3 * i + 1].strip()
        if el not in df["Material"].values:
            new_crystals.append([tokens[3 * i + 1], structure_type, float(tokens[3 * i + 2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])


# data = add_new(additional_fcc, "FCC", data)
# data = add_new(additional_bcc, "BCC", data)
# data.set_index("Material", inplace=True)
print(data)

       Material     a (Å)  Crystal structure    Ref.
0   C (diamond)  3.567000      Diamond (FCC)     [7]
2            Si  5.431021      Diamond (FCC)  [8][9]
3            Ge  5.658000      Diamond (FCC)     [8]
4          AlAs  5.660500  Zinc blende (FCC)     [8]
5           AlP  5.451000  Zinc blende (FCC)     [8]
..          ...       ...                ...     ...
79        KTaO3  3.988500   Cubic perovskite    [15]
81       SrTiO3  3.988050   Cubic perovskite    [15]
84       EuTiO3  7.810000   Cubic perovskite    [15]
85        SrVO3  3.838000   Cubic perovskite    [15]
86        CaVO3  3.767000   Cubic perovskite    [15]

[73 rows x 4 columns]


<ipython-input-6-08c8062c8bd3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)
<ipython-input-6-08c8062c8bd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["a (Å)"] = data["a (Å)"].map(float)


Setting 5 Angstrom lattice constant to all crystals

In [8]:
MAPI_KEY_LEGACY = '' #Optional, keep your MAPI_LEGACY_KEY here
predicted = []
mp = []
initial_structs = []
final_m3gnet_structs = []

mpr = MPRester(MAPI_KEY_LEGACY)

relaxer = Relaxer()  # This loads the default pre-trained model

# warnings.filterwarnings(action="ignore", category=UserWarning, module="tensorflow")
currently_m3gnet_failing = ['CsCl','CsI','Fe','Mo','W','V','Nb','Ta','Ne','CaVO3','Xe','Ar','Yb','Kr','Sr','Ac','Li','Na','Cs','K','Ba','Eu','Cr','Rb',"NC0.99"]

#for formula, v in tqdm(data.iterrows(), total=len(data)):
for ii,i in tqdm(data.iterrows(), total=len(data)):
  #M3gnet failing so skipping these
  formula=i['Material']
  if formula not in currently_m3gnet_failing: 
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    #cs = v["Crystal structure"]
    cs = i["Crystal structure"]
    wiki = i['a (Å)']
    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(
            Lattice.cubic(5),
            [els[0], els[1], els[2], els[2], els[2]],
            [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]],
        )
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0.0, 0.0, 0.0], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    print(s)
    initial_structs.append(s)
    relax_results = relaxer.relax(s)

    final_structure = relax_results["final_structure"]
    final_m3gnet_structs.append(final_structure)
    predicted.append(final_structure.lattice.a)
    
    #break
    try:
        mids = mpr.get_entries(s.composition.reduced_formula)
        for i in mids:
            try:
                structure = mpr.get_structure_by_material_id(i.entry_id)
                sga = SpacegroupAnalyzer(structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except Exception:
                pass
        else:
            raise RuntimeError
    except Exception:
        mp.append(0)
        traceback.print_exc()
    print('\nInitial,final,MP,Wiki,formula',5,final_structure.lattice.a,mp[-1],wiki)

  0%|          | 0/73 [00:00<?, ?it/s]

Full Formula (C8)
Reduced Formula: C
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  C     0.25  0.25  0.75
  1  C     0     0.5   0
  2  C     0.75  0.75  0.75
  3  C     0.5   0.5   0.5
  4  C     0.75  0.25  0.25
  5  C     0.5   0     0
  6  C     0     0     0.5
  7  C     0.25  0.75  0.25


  1%|▏         | 1/73 [00:40<49:02, 40.86s/it]


Initial,final,MP,Wiki,formula 5 3.571803997775669 3.57371 3.567
Full Formula (Si8)
Reduced Formula: Si
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.25  0.25  0.75
  1  Si    0     0.5   0
  2  Si    0.75  0.75  0.75
  3  Si    0.5   0.5   0.5
  4  Si    0.75  0.25  0.25
  5  Si    0.5   0     0
  6  Si    0     0     0.5
  7  Si    0.25  0.75  0.25


  3%|▎         | 2/73 [00:42<20:52, 17.64s/it]

...


Initial,final,MP,Wiki,formula 5 3.944806606480238 3.9451300000000002 3.98805
Full Formula (Eu1 Ti1 O3)
Reduced Formula: EuTiO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Eu    0    0    0
  1  Ti    0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


 97%|█████████▋| 71/73 [03:16<00:03,  1.81s/it]


Initial,final,MP,Wiki,formula 5 3.9294272674346593 3.9611945283278978 7.81
Full Formula (Sr1 V1 O3)
Reduced Formula: SrVO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Sr    0    0    0
  1  V     0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


100%|██████████| 73/73 [03:19<00:00,  2.73s/it]


Initial,final,MP,Wiki,formula 5 3.9060436247846058 3.900891 3.838


In [9]:
data_1 = data[~data["Material"].isin(currently_m3gnet_failing)]

In [10]:
data_1

,Material,a (Å),Crystal structure,Ref.
0,C (diamond),3.567000,Diamond (FCC),[7]
2,Si,5.431021,Diamond (FCC),[8][9]
3,Ge,5.658000,Diamond (FCC),[8]
4,AlAs,5.660500,Zinc blende (FCC),[8]
5,AlP,5.451000,Zinc blende (FCC),[8]
...,...,...,...,...
77,ScN,4.520000,Halite,[14]
79,KTaO3,3.988500,Cubic perovskite,[15]
81,SrTiO3,3.988050,Cubic perovskite,[15]
84,EuTiO3,7.810000,Cubic perovskite,[15]


In [11]:
data_1["MP a (Å)"] = mp
data_1["Predicted a (Å)"] = predicted
data_1["initial_structs"] = initial_structs
data_1["final_m3gnet_structs"] = final_m3gnet_structs

<ipython-input-11-02a3f8920e94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["MP a (Å)"] = mp
<ipython-input-11-02a3f8920e94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["Predicted a (Å)"] = predicted
<ipython-input-11-02a3f8920e94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [12]:
from sklearn.metrics import mean_absolute_error
print('MAE wrt Exp/Wiki data',mean_absolute_error(data_1['a (Å)'],data_1['Predicted a (Å)']))
print('MAE wrt MP data',mean_absolute_error(data_1['MP a (Å)'],data_1['Predicted a (Å)']))

MAE wrt Exp/Wiki data 0.1316390596491395
MAE wrt MP data 0.07474795094537603


In [13]:
from jarvis.core.atoms import Atoms
from alignn.ff.ff import ForceField, default_path
from jarvis.core.atoms import pmg_to_atoms
model_path = default_path()
def optimize_atoms(atoms=[],model_path=model_path):
    atoms = pmg_to_atoms(atoms)
    print(atoms.composition.reduced_formula)
    ff = ForceField(
        jarvis_atoms=atoms,
        model_path=model_path,
    )
    opt, en, fs = ff.optimize_atoms()
    return opt.lattice.a

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
model_path /usr/local/lib/python3.9/dist-packages/alignn/ff


In [ ]:
tqdm.pandas()
#slow
data_1['alignn_a']=data_1['initial_structs'].progress_apply(lambda x:optimize_atoms(atoms=x))

  0%|          | 0/63 [00:00<?, ?it/s]

C
OPTIMIZATION


/usr/local/lib/python3.9/dist-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-17.14335 eV KE= 0.00000 eV T= 0.000 K 
a= 4.939 Ang b= 4.939 Ang c= 4.939 Ang Volume= 120.491 amu/a3 PE=-17.95720 eV KE= 0.00000 eV T= 0.000 K 
a= 4.817 Ang b= 4.817 Ang c= 4.816 Ang Volume= 111.751 amu/a3 PE=-19.64051 eV KE= 0.00000 eV T= 0.000 K 
a= 4.623 Ang b= 4.627 Ang c= 4.623 Ang Volume= 98.891 amu/a3 PE=-23.75317 eV KE= 0.00000 eV T= 0.000 K 
a= 4.316 Ang b= 4.333 Ang c= 4.316 Ang Volume= 80.709 amu/a3 PE=-32.82676 eV KE= 0.00000 eV T= 0.000 K 
a= 3.943 Ang b= 3.987 Ang c= 3.943 Ang Volume= 61.972 amu/a3 PE=-46.12150 eV KE= 0.00000 eV T= 0.000 K 
a= 3.513 Ang b= 3.578 Ang c= 3.513 Ang Volume= 44.078 amu/a3 PE=-61.28164 eV KE= 0.00000 eV T= 0.000 K 
a= 3.513 Ang b= 3.578 Ang c= 3.513 Ang Volume= 44.090 amu/a3 PE=-61.28210 eV KE= 0.00000 eV T= 0.000 K 
a= 3.514 Ang b= 3.579 Ang c= 3.514 Ang Volume= 44.113 amu/a3 PE=-61.28297 eV KE= 0.00000 eV T= 0.000 K 
a= 3.515 Ang b= 3.579 Ang c= 3.515 Ang Volume= 44.148 amu/a3 

In [15]:
pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
alignn==2023.4.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
ase==3.22.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
dgl==1.